贾轩茗 金融学 202111030026

In [ ]:
pip install feature_engine

In [1]:
import numpy as np
import pandas as pd
import sklearn
import feature_engine
import seaborn as sns
from pandas import Series,DataFrame
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler # 标准化
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV, ShuffleSplit, cross_val_score, cross_validate
from sklearn.compose import ColumnTransformer
from feature_engine.encoding import OneHotEncoder # 分类变量数值化

In [2]:
train = pd.read_csv('/kaggle/input/classification/asap-bnu-train-4.csv')
print(train.shape)
test = pd.read_csv('/kaggle/input/classification/asap-bnu-test-4-features.csv')
print(test.shape)

(1416, 28)
(354, 27)


In [4]:
train.head()

,essay_id,score,genre,sent_num,max_depth,mean_depth,num_word,num_unique_word,num_noun,num_unique_noun,...,num_special_token,num_unique_special_token,num_unique_lemma,mean_word_length,mean_sent_length,bigram_lemma_ttr,trigram_lemma_ttr,task_fulfillment,depth_variation_level,error_density
0,DMsvCCW,0,source-based,2,4,4.000000,24.0,17.0,2.0,2.0,...,2.0,1.0,17.0,3.083333,11.000000,0.954545,1.000000,partially,small,medium
1,Oq55l8p,1,source-based,1,1,1.000000,2.0,2.0,1.0,1.0,...,0.0,0.0,2.0,3.000000,2.000000,1.000000,1.000000,partially,small,low
2,cDXkupP,2,source-based,7,8,5.857143,150.0,85.0,27.0,19.0,...,13.0,2.0,84.0,3.233333,22.666667,0.897810,0.977941,partially,large,medium
3,6wPJQmo,2,source-based,9,7,4.888889,145.0,79.0,22.0,16.0,...,9.0,1.0,73.0,3.255172,15.000000,0.888060,0.962406,partially,large,low
4,zJzoiGC,3,source-based,10,8,5.100000,230.0,100.0,36.0,24.0,...,16.0,2.0,90.0,3.247826,21.400000,0.812207,0.933962,mostly,large,low


In [5]:
# 区分分类和连续变量
continuous_cols = train.columns[3:25]
categorical_cols = train.columns[25:].append(train.columns[2:3])
print(continuous_cols)
print(categorical_cols)

Index(['sent_num', 'max_depth', 'mean_depth', 'num_word', 'num_unique_word',
       'num_noun', 'num_unique_noun', 'num_verb', 'num_unique_verb', 'num_adj',
       'num_unique_adj', 'num_adv', 'num_unique_adv', 'num_pron',
       'num_unique_pron', 'num_special_token', 'num_unique_special_token',
       'num_unique_lemma', 'mean_word_length', 'mean_sent_length',
       'bigram_lemma_ttr', 'trigram_lemma_ttr'],
      dtype='object')
Index(['task_fulfillment', 'depth_variation_level', 'error_density', 'genre'], dtype='object')


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_train = pd.DataFrame([])
label_test = pd.DataFrame([])

for feature in categorical_cols:
    le.fit(train[feature])
    label_train[feature] = le.transform(train[feature])
    label_test[feature] = le.transform(test[feature])
    
label_train.head()

,task_fulfillment,depth_variation_level,error_density,genre
0,3,1,2,0
1,3,1,1,0
2,3,0,2,0
3,3,0,1,0
4,2,0,1,0


In [7]:
scaler = StandardScaler()
scale_train = scaler.fit_transform(pd.DataFrame(train[continuous_cols]))
scale_test = scaler.fit_transform(pd.DataFrame(test[continuous_cols]))
scale_train_df = pd.DataFrame(scale_train, columns=continuous_cols)
scale_test_df = pd.DataFrame(scale_test, columns=continuous_cols)
scale_train_df.head()

,sent_num,max_depth,mean_depth,num_word,num_unique_word,num_noun,num_unique_noun,num_verb,num_unique_verb,num_adj,...,num_unique_adv,num_pron,num_unique_pron,num_special_token,num_unique_special_token,num_unique_lemma,mean_word_length,mean_sent_length,bigram_lemma_ttr,trigram_lemma_ttr
0,-1.002190,-1.628956,-0.908143,-1.399836,-1.660632,-1.509268,-1.626923,-1.492251,-1.513569,-0.863678,...,-1.098435,-0.947649,-1.427843,-0.986592,-1.016585,-1.669891,-1.176287,-1.143656,0.639648,0.683305
1,-1.343730,-3.079649,-2.959450,-1.780277,-2.235045,-1.603522,-1.770933,-1.842379,-1.932029,-1.114389,...,-1.424511,-1.664230,-2.275569,-1.314799,-1.870371,-2.285503,-1.752763,-2.083700,1.395845,0.683305
2,0.705513,0.305301,0.361714,0.779054,0.943373,0.847091,0.821242,0.491812,0.439245,0.891298,...,0.858023,1.345412,1.539197,0.818548,-0.162798,1.079843,-0.138631,0.074919,-0.304218,0.120557
3,1.388593,-0.178263,-0.300348,0.692590,0.713608,0.375819,0.389213,0.608522,0.718219,1.142009,...,-0.120206,0.485514,0.691471,0.162133,-1.016585,0.628394,0.012445,-0.725859,-0.466431,-0.275765
4,1.730134,0.305301,-0.155997,2.162476,1.517786,1.695380,1.541290,2.009037,1.415652,1.894142,...,1.184099,0.915463,0.691471,1.310858,-0.162798,1.326087,-0.038374,-0.057384,-1.728348,-1.001400


In [8]:
train_features = pd.concat([scale_train_df, label_train], axis=1)
test_features = pd.concat([scale_test_df, label_test], axis=1)
print(train_features.shape)
print(test_features.shape)

(1416, 26)
(354, 26)


In [9]:
train_score = train['score']
train1 = pd.concat([train_score,train_features], axis=1)
train_data, validate_data = train_test_split(train1, test_size=0.2, random_state=42)

In [10]:
train_data.describe()

,score,sent_num,max_depth,mean_depth,num_word,num_unique_word,num_noun,num_unique_noun,num_verb,num_unique_verb,...,num_unique_special_token,num_unique_lemma,mean_word_length,mean_sent_length,bigram_lemma_ttr,trigram_lemma_ttr,task_fulfillment,depth_variation_level,error_density,genre
count,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,...,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.0
mean,1.432862,-0.022825,-0.017217,0.010287,-0.013786,-0.019158,-0.007526,-0.011012,-0.015545,-0.017290,...,-0.011953,-0.018796,0.011319,0.021987,-0.004564,-0.003620,2.500000,0.526502,1.387809,0.0
std,0.947750,0.994541,0.997587,1.026220,0.990746,0.989119,0.993379,0.987297,0.989015,0.986849,...,1.003577,0.989052,1.006895,1.021440,1.018620,1.042801,0.952914,0.499518,0.764342,0.0
min,0.000000,-1.343730,-3.079649,-2.959450,-1.780277,-2.235045,-1.697776,-1.914943,-1.842379,-1.932029,...,-1.870371,-2.285503,-8.737634,-2.083700,-7.920499,-11.984346,0.000000,0.000000,0.000000,0.0
25%,1.000000,-0.660649,-0.661827,-0.634635,-0.794588,-0.818160,-0.755233,-0.762865,-0.791993,-0.816135,...,-1.016585,-0.808034,-0.429596,-0.621410,-0.543686,-0.290407,2.000000,0.000000,1.000000,0.0
50%,1.000000,-0.319109,-0.178263,-0.148399,-0.154756,-0.109717,-0.189707,-0.186826,-0.208445,-0.118702,...,-0.162798,-0.110341,0.021008,-0.203613,0.100708,0.284692,3.000000,1.000000,2.000000,0.0
75%,2.000000,0.705513,0.305301,0.459396,0.640712,0.637020,0.564328,0.677232,0.608522,0.578732,...,0.690988,0.628394,0.585472,0.353450,0.703562,0.683305,3.000000,1.000000,2.000000,0.0
max,3.000000,5.487078,6.591636,6.613318,3.805290,3.470790,3.863230,4.133465,4.226519,4.065900,...,4.959920,3.419168,3.781404,8.465678,1.395845,0.683305,3.000000,1.000000,2.000000,0.0


In [ ]:
correlation_matrix = train_data.corr()
plt.figure(figsize=(20, 18))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(train_data['score'])
plt.title("Train data - score")
plt.show()

In [ ]:
train_data.hist(figsize=(12, 10), color='blue', edgecolor='black',bins=50)
plt.gcf().set_facecolor('lightgrey')
plt.tight_layout()
plt.show()

In [13]:
selected_features=[
    'num_word',
    'bigram_lemma_ttr',
    'mean_sent_length',
    'task_fulfillment',    
    'depth_variation_level'
]

In [12]:
selected_features = train_features.columns
selected_features = selected_features.drop('genre')
selected_features

Index(['sent_num', 'max_depth', 'mean_depth', 'num_word', 'num_unique_word',
       'num_noun', 'num_unique_noun', 'num_verb', 'num_unique_verb', 'num_adj',
       'num_unique_adj', 'num_adv', 'num_unique_adv', 'num_pron',
       'num_unique_pron', 'num_special_token', 'num_unique_special_token',
       'num_unique_lemma', 'mean_word_length', 'mean_sent_length',
       'bigram_lemma_ttr', 'trigram_lemma_ttr', 'task_fulfillment',
       'depth_variation_level', 'error_density'],
      dtype='object')

In [14]:
train_data_use = train_data[selected_features]
validate_data_use = validate_data[selected_features]
test_data_use = test_features[selected_features]
train_data_use.head()

,num_word,bigram_lemma_ttr,mean_sent_length,task_fulfillment,depth_variation_level
767,-0.293098,0.771983,-1.083971,3,0
486,-1.469007,1.395845,-0.308062,3,1
48,1.626400,-0.414325,2.694855,3,1
155,-0.932931,0.716811,0.214184,0,0
584,-0.984809,1.395845,2.512069,3,1


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier

In [16]:
# 定义分类模型
logistic_model = LogisticRegression(random_state=42, multi_class='auto')
rf_model = RandomForestClassifier(random_state=42)
decisiontree_model = DecisionTreeClassifier(random_state=42)
svm_model = OneVsRestClassifier(SVC(probability=True))
naiveB_model = GaussianNB()
gbtree_model = GradientBoostingClassifier(random_state=42)
nn_model = MLPClassifier(random_state=42)
xgb_model = xgb.XGBClassifier(random_state=42)

In [17]:
from sklearn.metrics import cohen_kappa_score

# 重新定义交叉验证函数，计算QWK分数
def cv_scores(model, x, y, cv=10):
    cv_scores = cross_val_score(model, x, y, cv=cv, scoring='accuracy')
    qwk_scores = []
    for i, (train_index, test_index) in enumerate(KFold(n_splits=cv).split(x)):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        qwk_score = cohen_kappa_score(y_test, pred, weights='quadratic')
        qwk_scores.append(qwk_score)
        print(f"CV {i+1} QWK Score: {qwk_score}")
    return qwk_scores

In [ ]:
model_list = [
    logistic_model,
    rf_model,
    decisiontree_model, 
    svm_model, 
    naiveB_model,
    gbtree_model,
    nn_model,
    xgb_model
]

best_model = None
best_qwk = 0.0

for model in model_list:
    model.fit(train_data_use, train_data['score'])
    print(str(model) + "训练集表现：")
    cv_scores(model, train_data_use, train_data['score'])
    pred = model.predict(validate_data_use)
    qwk = cohen_kappa_score(validate_data['score'], pred, weights='quadratic')
    accuracy = accuracy_score(validate_data['score'], pred)
    print(str(model) + "验证集表现：")
    print("QWK Score:", qwk)
    print("Accuracy:", accuracy)
    
    if qwk > best_qwk:
        best_qwk = qwk
        best_model = model

print("最佳模型：", best_model)
print("最佳模型验证集QWK：", best_qwk)

### 参数调优

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score

# 重新定义QWK评分函数
def qwk_scorer(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

logistic_model = LogisticRegression(random_state=42, multi_class='auto')
param_grid = {
    'C': [0.1, 1, 10],  # 正则化参数
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # 求解器
    'max_iter': [100, 200, 300]  # 最大迭代次数
}

# 交叉验证获得最佳参数

grid_search_logistic = GridSearchCV(estimator=logistic_model, param_grid=param_grid, cv=10, scoring=make_scorer(qwk_scorer))
grid_search_logistic.fit(train_data_use, train_data['score'])
print("最佳参数：", grid_search_logistic.best_params_)
print("最佳得分：", grid_search_logistic.best_score_)

# 使用最佳参数的模型进行预测
best_logistic_model = grid_search_logistic.best_estimator_
pred_logistic = best_logistic_model.predict(validate_data_use)

# 计算QWK
qwk_logistic = qwk_scorer(validate_data['score'], pred_logistic)
print("Logistic回归验证集QWK：", qwk_logistic)


最佳参数： {'C': 1, 'max_iter': 100, 'solver': 'liblinear'}
最佳得分： 0.7328732196413743
Logistic回归验证集QWK： 0.7056801171226004


In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc_model = SVC()

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

grid_search_svc = GridSearchCV(estimator=svc_model, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search_svc.fit(train_data_use, train_data['score'])
print("最佳参数：", grid_search_svc.best_params_)
print("最佳得分：", grid_search_svc.best_score_)

# 使用最佳参数的模型进行预测
best_svc_model = grid_search_svc.best_estimator_
pred_svc = best_svc_model.predict(validate_data_use)
accuracy_svc = accuracy_score(validate_data['score'], pred_svc)
print("svc验证集表现：", accuracy_svc)

最佳参数： {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
最佳得分： 0.666953889147648
svc验证集表现： 0.6514084507042254


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

svc_model = SVC()

ovr_model = OneVsRestClassifier(svc_model)

pipeline = Pipeline([('classifier', ovr_model)])

param_grid = {
    'classifier__estimator__C': [0.1, 1, 10],
    'classifier__estimator__kernel': ['linear', 'rbf', 'poly'],
    'classifier__estimator__gamma': ['scale', 'auto']
}

grid_search_svc = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search_svc.fit(train_data_use, train_data['score'])
print("最佳参数：", grid_search_svc.best_params_)
print("最佳得分：", grid_search_svc.best_score_)

# 使用最佳参数的模型进行预测
best_svc_model = grid_search_svc.best_estimator_
pred_svc = best_svc_model.predict(validate_data_use)
accuracy_svc = accuracy_score(validate_data['score'], pred_svc)
print("svc验证集表现：", accuracy_svc)

In [ ]:
from sklearn.metrics import make_scorer

gb_model = GradientBoostingClassifier()

param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5]
}

grid_search_gb = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=10, scoring=make_scorer(qwk_scorer))
grid_search_gb.fit(train_data_use, train_data['score'])
print("最佳参数：", grid_search_gb.best_params_)
print("最佳得分：", grid_search_gb.best_score_)

best_gb_model = grid_search_gb.best_estimator_
pred_gb = best_gb_model.predict(validate_data_use)
qwk_gb = qwk_scorer(validate_data['score'], pred_gb)
print("梯度提升验证集QWK：", qwk_gb)

In [ ]:
xgb_clf = xgb.XGBClassifier()

param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],  # 学习率
    'n_estimators': [50, 100, 200],  # 迭代次数
    'max_depth': [3, 4, 5],  # 树的最大深度
    'min_child_weight': [1, 2, 3]  # 叶子节点最小权重
}

grid_search_xgb = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search_xgb.fit(train_data_use, train_data['score'])
print("最佳参数：", grid_search_xgb.best_params_)
print("最佳得分：", grid_search_xgb.best_score_)

# 使用最佳参数的模型进行预测
best_xgb_model = grid_search_xgb.best_estimator_
y_pred = best_xgb_model.predict(validate_data_use)
accuracy = accuracy_score(validate_data['score'], y_pred)
print("验证集表现：", accuracy)

### 测试集预测

In [19]:
pred_test = best_svc_model.predict(test_data_use)

In [20]:
output_dict = {
    'essay_id': test['essay_id'],
    'score': pred_test
}
output_df = pd.DataFrame(output_dict)
output_df.to_csv("/kaggle/working/submission_svc.csv", index=False)